In [77]:

# load database data.db into pandas dataframe
import pandas as pd
import os
import sqlite3
import sys
import pandas as pd

In [78]:
# load all tables from database withouth a function

# connect to database
conn = sqlite3.connect('data.db')

# load all tables from database
data_df = pd.read_sql_query("SELECT * FROM data", conn)
event_data_df = pd.read_sql_query("SELECT * FROM events", conn)

In [79]:
# drop the columns: location_name, temperature, weather_condition, unverified
data_df = data_df.drop(columns=['location_name', 'temperature', 'weather_condition', 'unverified'])
data_df.sort_values(by=['timestamp', 'location_id'], inplace=True)
data_df

,pedestrians_count,timestamp,location_id
6110,1346,2024-01-01T00:00:00+01:00,135.0
6130,388,2024-01-01T00:00:00+01:00,470.0
6150,1197,2024-01-01T00:00:00+01:00,476.0
6111,772,2024-01-01T01:00:00+01:00,135.0
6131,161,2024-01-01T01:00:00+01:00,470.0
...,...,...,...
13784,951,2024-07-11T18:00:00+02:00,470.0
13785,1784,2024-07-11T18:00:00+02:00,476.0
13786,1469,2024-07-11T19:00:00+02:00,135.0
13787,853,2024-07-11T19:00:00+02:00,470.0


In [80]:
# change column name datum to timestamp
event_data_df.head()


,date,holiday,workday,green_market,specialities_market,event
0,2024-01-01,1,1,0,0,0
1,2024-01-02,0,1,1,1,0
2,2024-01-03,0,1,1,1,0
3,2024-01-04,0,1,0,0,0
4,2024-01-05,0,1,1,1,0


In [81]:
# add a column named date to data_df from timestamp using regex
data_df['date'] = data_df['timestamp'].str.extract(r'(\d{4}-\d{2}-\d{2})')

In [82]:
data_df.head()

,pedestrians_count,timestamp,location_id,date
6110,1346,2024-01-01T00:00:00+01:00,135.0,2024-01-01
6130,388,2024-01-01T00:00:00+01:00,470.0,2024-01-01
6150,1197,2024-01-01T00:00:00+01:00,476.0,2024-01-01
6111,772,2024-01-01T01:00:00+01:00,135.0,2024-01-01
6131,161,2024-01-01T01:00:00+01:00,470.0,2024-01-01


In [83]:
# merge the two dataframes on the date (string) column
merged_df = pd.merge(data_df, event_data_df, on='date', how='left') # left join to keep all rows from data_df
merged_df.head()

,pedestrians_count,timestamp,location_id,date,holiday,workday,green_market,specialities_market,event
0,1346,2024-01-01T00:00:00+01:00,135.0,2024-01-01,1,1,0,0,0
1,388,2024-01-01T00:00:00+01:00,470.0,2024-01-01,1,1,0,0,0
2,1197,2024-01-01T00:00:00+01:00,476.0,2024-01-01,1,1,0,0,0
3,772,2024-01-01T01:00:00+01:00,135.0,2024-01-01,1,1,0,0,0
4,161,2024-01-01T01:00:00+01:00,470.0,2024-01-01,1,1,0,0,0


In [84]:
# add time colums to the merged dataframe: year, month, day, hour, weekday using regex
merged_df['year'] = merged_df['timestamp'].str.extract(r'(\d{4})').astype(int)
merged_df['month'] = merged_df['timestamp'].str.extract(r'-(\d{2})-').astype(int)
merged_df['day'] = merged_df['timestamp'].str.extract(r'-(\d{2})T').astype(int)
merged_df['hour'] = merged_df['timestamp'].str.extract(r'T(\d{2})').astype(int)
merged_df['weekday'] = pd.to_datetime(merged_df['date']).dt.day_name()
merged_df.head()

,pedestrians_count,timestamp,location_id,date,holiday,workday,green_market,specialities_market,event,year,month,day,hour,weekday
0,1346,2024-01-01T00:00:00+01:00,135.0,2024-01-01,1,1,0,0,0,2024,1,1,0,Monday
1,388,2024-01-01T00:00:00+01:00,470.0,2024-01-01,1,1,0,0,0,2024,1,1,0,Monday
2,1197,2024-01-01T00:00:00+01:00,476.0,2024-01-01,1,1,0,0,0,2024,1,1,0,Monday
3,772,2024-01-01T01:00:00+01:00,135.0,2024-01-01,1,1,0,0,0,2024,1,1,1,Monday
4,161,2024-01-01T01:00:00+01:00,470.0,2024-01-01,1,1,0,0,0,2024,1,1,1,Monday


In [85]:
merged_df.drop(columns=['date'], inplace=True)
#merged_df.drop(columns=['timestamp'], inplace=True)
merged_df.head()

,pedestrians_count,timestamp,location_id,holiday,workday,green_market,specialities_market,event,year,month,day,hour,weekday
0,1346,2024-01-01T00:00:00+01:00,135.0,1,1,0,0,0,2024,1,1,0,Monday
1,388,2024-01-01T00:00:00+01:00,470.0,1,1,0,0,0,2024,1,1,0,Monday
2,1197,2024-01-01T00:00:00+01:00,476.0,1,1,0,0,0,2024,1,1,0,Monday
3,772,2024-01-01T01:00:00+01:00,135.0,1,1,0,0,0,2024,1,1,1,Monday
4,161,2024-01-01T01:00:00+01:00,470.0,1,1,0,0,0,2024,1,1,1,Monday


In [86]:
merged_df.dtypes

pedestrians_count        int64
timestamp               object
location_id            float64
holiday                  int64
workday                  int64
green_market             int64
specialities_market      int64
event                    int64
year                     int32
month                    int32
day                      int32
hour                     int32
weekday                 object
dtype: object

In [87]:
# strip the timezone from the timestamp column: datetime_str.split('+')[0]
merged_df['datetime'] = merged_df['timestamp'].str.split('+').str[0]
merged_df

,pedestrians_count,timestamp,location_id,holiday,workday,green_market,specialities_market,event,year,month,day,hour,weekday,datetime
0,1346,2024-01-01T00:00:00+01:00,135.0,1,1,0,0,0,2024,1,1,0,Monday,2024-01-01T00:00:00
1,388,2024-01-01T00:00:00+01:00,470.0,1,1,0,0,0,2024,1,1,0,Monday,2024-01-01T00:00:00
2,1197,2024-01-01T00:00:00+01:00,476.0,1,1,0,0,0,2024,1,1,0,Monday,2024-01-01T00:00:00
3,772,2024-01-01T01:00:00+01:00,135.0,1,1,0,0,0,2024,1,1,1,Monday,2024-01-01T01:00:00
4,161,2024-01-01T01:00:00+01:00,470.0,1,1,0,0,0,2024,1,1,1,Monday,2024-01-01T01:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13784,951,2024-07-11T18:00:00+02:00,470.0,0,1,0,0,1,2024,7,11,18,Thursday,2024-07-11T18:00:00
13785,1784,2024-07-11T18:00:00+02:00,476.0,0,1,0,0,1,2024,7,11,18,Thursday,2024-07-11T18:00:00
13786,1469,2024-07-11T19:00:00+02:00,135.0,0,1,0,0,1,2024,7,11,19,Thursday,2024-07-11T19:00:00
13787,853,2024-07-11T19:00:00+02:00,470.0,0,1,0,0,1,2024,7,11,19,Thursday,2024-07-11T19:00:00


In [88]:
# load weather data from database
weather_df = pd.read_sql_query("SELECT datetime, temp, humidity, precip FROM weather", conn)
weather_df.head()

,datetime,temp,humidity,precip
0,2024-01-01T00:00:00,4.8,87.50,0.0
1,2024-01-01T01:00:00,4.9,80.73,0.0
2,2024-01-01T02:00:00,6.2,75.01,0.0
3,2024-01-01T03:00:00,6.2,75.55,0.0
4,2024-01-01T04:00:00,5.7,79.25,0.0


In [91]:
# merge the weather data with the merged_df on the timestamp column
final_df = pd.merge(merged_df, weather_df, on='datetime', how='left')
final_df

,pedestrians_count,timestamp,location_id,holiday,workday,green_market,specialities_market,event,year,month,day,hour,weekday,datetime,temp,humidity,precip
0,1346,2024-01-01T00:00:00+01:00,135.0,1,1,0,0,0,2024,1,1,0,Monday,2024-01-01T00:00:00,4.8,87.50,0.0
1,388,2024-01-01T00:00:00+01:00,470.0,1,1,0,0,0,2024,1,1,0,Monday,2024-01-01T00:00:00,4.8,87.50,0.0
2,1197,2024-01-01T00:00:00+01:00,476.0,1,1,0,0,0,2024,1,1,0,Monday,2024-01-01T00:00:00,4.8,87.50,0.0
3,772,2024-01-01T01:00:00+01:00,135.0,1,1,0,0,0,2024,1,1,1,Monday,2024-01-01T01:00:00,4.9,80.73,0.0
4,161,2024-01-01T01:00:00+01:00,470.0,1,1,0,0,0,2024,1,1,1,Monday,2024-01-01T01:00:00,4.9,80.73,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13784,951,2024-07-11T18:00:00+02:00,470.0,0,1,0,0,1,2024,7,11,18,Thursday,2024-07-11T18:00:00,28.0,45.09,0.1
13785,1784,2024-07-11T18:00:00+02:00,476.0,0,1,0,0,1,2024,7,11,18,Thursday,2024-07-11T18:00:00,28.0,45.09,0.1
13786,1469,2024-07-11T19:00:00+02:00,135.0,0,1,0,0,1,2024,7,11,19,Thursday,2024-07-11T19:00:00,27.0,50.97,0.0
13787,853,2024-07-11T19:00:00+02:00,470.0,0,1,0,0,1,2024,7,11,19,Thursday,2024-07-11T19:00:00,27.0,50.97,0.0


In [90]:
# drop the datetime column
final_df.drop(columns=['datetime'], inplace=True)
final_df.drop(columns=['timestamp'], inplace=True)
final_df

KeyError: "['datetime'] not found in axis"